# <center>NLP with french tweets dataset</center>

In [2]:
from datetime import datetime
start_time0 = datetime.now()
import spacy
import numpy as np
import re
import nltk
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pickle
import xgboost as xgb
from ipynb.fs.full.my_functions import clean_up, stem_spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk import NaiveBayesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

## Step 1 :  ML model construction 

In [3]:
train= pd.read_csv('tweets.csv',index_col=[0]).reset_index(drop=True)
train.head()

,polarity,statutnull
0,0,"Un avertissement juste: en raison de ce refus obstiné d'envisager des exemptions sans but lucratif, nous devons ne pas suivre ceux qui ne nous suivent pas."
1,4,merci!
2,0,"Réussi à écrire quelque chose en valeur! Au lit maintenant. J'espère que les musiques resteront ou bien, je pourrais pleurer."
3,4,En allant à kroger puis à la piscine. Attendant un thé à mcdon.
4,4,c'est génial


In [4]:
train = train.reset_index(drop=True)

In [5]:
train.head()

,polarity,statutnull
0,0,"Un avertissement juste: en raison de ce refus obstiné d'envisager des exemptions sans but lucratif, nous devons ne pas suivre ceux qui ne nous suivent pas."
1,4,merci!
2,0,"Réussi à écrire quelque chose en valeur! Au lit maintenant. J'espère que les musiques resteront ou bien, je pourrais pleurer."
3,4,En allant à kroger puis à la piscine. Attendant un thé à mcdon.
4,4,c'est génial


In [6]:
train.columns

Index(['polarity', 'statutnull'], dtype='object')

In [7]:

train = train.dropna()
train['polarity'] = train['polarity'].str.replace("0","negatif").str.replace("4","positif")
index_zero = train[(train['polarity'] != 'positif') & (train['polarity']!= 'negatif')].index
train.drop(index_zero, inplace=True)

In [8]:
train['polarity'].value_counts()
#.plot(kind='bar')

negatif    149815
positif    149583
Name: polarity, dtype: int64

In [9]:
from ipynb.fs.full.my_functions import clean_up
from ipynb.fs.full.my_functions import stem_spacy
train.statutnull = train.statutnull.astype(str)
train['original'] = train['statutnull']
train.statutnull = train.statutnull.apply(lambda s: clean_up(s))
train['tweet_stem'] = stem_spacy(train.statutnull)
train.rename(columns={'statutnull':'tweet'},inplace=True)
train.head()

,polarity,tweet,original,tweet_stem
0,negatif,avertissement juste raison refus obstiné d'envisager exemptions sans but lucratif devons suivre ceux suivent,"Un avertissement juste: en raison de ce refus obstiné d'envisager des exemptions sans but lucratif, nous devons ne pas suivre ceux qui ne nous suivent pas.","[avertissement, juste, raison, refus, obstiner, de, envisager, exemption, sans, boire, lucratif, devon, suivre, celui, suivre]"
1,positif,merci,merci!,[merci]
2,negatif,réussi écrire quelque chose valeur lit maintenant j'espère musiques resteront bien pourrais pleurer,"Réussi à écrire quelque chose en valeur! Au lit maintenant. J'espère que les musiques resteront ou bien, je pourrais pleurer.","[réussir, écrire, quelque, chose, valeur, lire, maintenir, il, espérer, musiqu, rester, bien, pouvoir, pleurer]"
3,positif,allant kroger puis piscine attendant thé mcdon,En allant à kroger puis à la piscine. Attendant un thé à mcdon.,"[aller, kroger, pouvoir, piscine, attendre, thé, mcdon]"
4,positif,c'est génial,c'est génial,"[ce, être, génial]"


In [10]:
train.shape

(299398, 4)

In [11]:
train.reset_index(drop=True).head()

,polarity,tweet,original,tweet_stem
0,negatif,avertissement juste raison refus obstiné d'envisager exemptions sans but lucratif devons suivre ceux suivent,"Un avertissement juste: en raison de ce refus obstiné d'envisager des exemptions sans but lucratif, nous devons ne pas suivre ceux qui ne nous suivent pas.","[avertissement, juste, raison, refus, obstiner, de, envisager, exemption, sans, boire, lucratif, devon, suivre, celui, suivre]"
1,positif,merci,merci!,[merci]
2,negatif,réussi écrire quelque chose valeur lit maintenant j'espère musiques resteront bien pourrais pleurer,"Réussi à écrire quelque chose en valeur! Au lit maintenant. J'espère que les musiques resteront ou bien, je pourrais pleurer.","[réussir, écrire, quelque, chose, valeur, lire, maintenir, il, espérer, musiqu, rester, bien, pouvoir, pleurer]"
3,positif,allant kroger puis piscine attendant thé mcdon,En allant à kroger puis à la piscine. Attendant un thé à mcdon.,"[aller, kroger, pouvoir, piscine, attendre, thé, mcdon]"
4,positif,c'est génial,c'est génial,"[ce, être, génial]"


In [12]:
train['tweet_stem_join'] = train.tweet_stem.apply(lambda x: ' '.join(x)).astype(str)
train.head()

,polarity,tweet,original,tweet_stem,tweet_stem_join
0,negatif,avertissement juste raison refus obstiné d'envisager exemptions sans but lucratif devons suivre ceux suivent,"Un avertissement juste: en raison de ce refus obstiné d'envisager des exemptions sans but lucratif, nous devons ne pas suivre ceux qui ne nous suivent pas.","[avertissement, juste, raison, refus, obstiner, de, envisager, exemption, sans, boire, lucratif, devon, suivre, celui, suivre]",avertissement juste raison refus obstiner de envisager exemption sans boire lucratif devon suivre celui suivre
1,positif,merci,merci!,[merci],merci
2,negatif,réussi écrire quelque chose valeur lit maintenant j'espère musiques resteront bien pourrais pleurer,"Réussi à écrire quelque chose en valeur! Au lit maintenant. J'espère que les musiques resteront ou bien, je pourrais pleurer.","[réussir, écrire, quelque, chose, valeur, lire, maintenir, il, espérer, musiqu, rester, bien, pouvoir, pleurer]",réussir écrire quelque chose valeur lire maintenir il espérer musiqu rester bien pouvoir pleurer
3,positif,allant kroger puis piscine attendant thé mcdon,En allant à kroger puis à la piscine. Attendant un thé à mcdon.,"[aller, kroger, pouvoir, piscine, attendre, thé, mcdon]",aller kroger pouvoir piscine attendre thé mcdon
4,positif,c'est génial,c'est génial,"[ce, être, génial]",ce être génial


In [13]:
count_vectorizer = CountVectorizer(analyzer='word') 
cv = count_vectorizer.fit(train['tweet_stem_join'])
pickle.dump(cv, open("cv1.pkl", "wb"))

In [14]:
cv_ = count_vectorizer.fit_transform(train['tweet_stem_join'])
cv_.shape

(299398, 84662)

In [15]:
cv1 = pickle.load(open("cv1.pkl", 'rb'))
cv1_new = CountVectorizer(vocabulary = cv1.vocabulary_)
X_cv1 = cv1_new.transform(train['tweet_stem_join'])
X_cv1.shape

(299398, 84662)

In [16]:
end_time0 = datetime.now()
print('Time import and clean: {}'.format(end_time0 - start_time0))

Time import and clean: 1:35:16.193779


## SVM

In [17]:
start_time_svc = datetime.now()
X_train_svc,X_test_svc,y_train_svc,y_test_svc = train_test_split(cv_,train['polarity'],test_size=.2,
                                                 stratify=train['polarity'], random_state=42)

In [18]:
svc = SVC()
svc.fit(X_train_svc,y_train_svc)
prediction_svc = svc.predict(X_test_svc)
print(classification_report(prediction_svc,y_test_svc))
end_time_svc = datetime.now()
print('Temps modèle SVM : {}'.format(end_time_svc - start_time_svc))

C:\Users\aline.debenath\AppData\Local\Continuum\anaconda3\envs\spacy\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

     negatif       0.42      0.63      0.50     19969
     positif       0.75      0.56      0.65     39911

   micro avg       0.59      0.59      0.59     59880
   macro avg       0.59      0.60      0.57     59880
weighted avg       0.64      0.59      0.60     59880

Temps modèle SVM : 4:23:08.189041


## XGBoost model

In [19]:
start_time1 = datetime.now()
X_train_xg,X_test_xg,y_train_xg,y_test_xg = train_test_split(cv_,train['polarity'],test_size=.2,stratify=train['polarity'], random_state=42)

In [20]:
xgbc = XGBClassifier(max_depth=9, min_child_weight=4, n_estimators=1000, nthread= 3)
xgbc.fit(X_train_xg,y_train_xg)
prediction_xgb = xgbc.predict(X_test_xg)
print(classification_report(prediction_xgb,y_test_xg))
#0.742425 count vectorizer (max_depth=6, n_estimators=1000, nthread= 3)
#0.7415 tfidf
print(accuracy_score(prediction_xgb,y_test_xg))
#0.7531 300.000 sample 21/01
end_time1 = datetime.now()
print('Temps modèle XGBoost : {}'.format(end_time1 - start_time1))

              precision    recall  f1-score   support

     negatif       0.71      0.78      0.75     27366
     positif       0.80      0.74      0.77     32514

   micro avg       0.76      0.76      0.76     59880
   macro avg       0.76      0.76      0.76     59880
weighted avg       0.76      0.76      0.76     59880

0.7559285237140948
Temps modèle XGBoost : 10:36:01.214269


## Logistic regression model

In [21]:
start_time2 = datetime.now()
X_train_logreg,X_test_logreg,y_train_logreg,y_test_logreg = train_test_split(cv_,train['polarity'] , test_size=.2,
                                                 stratify=train['polarity'], random_state=42)

In [22]:
logreg = LogisticRegression(C=0.1,penalty="l2")
logreg.fit(X_train_logreg,y_train_logreg)
prediction_logreg = logreg.predict(X_test_logreg)
print(classification_report(prediction_logreg,y_test_logreg))
#0.7628 count vectorizer
#0.74105 tfidf
print(accuracy_score(prediction_logreg,y_test_logreg))
end_time2 = datetime.now()
print('Time Logistic regression model: {}'.format(end_time2 - start_time2))

C:\Users\aline.debenath\AppData\Local\Continuum\anaconda3\envs\spacy\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

     negatif       0.73      0.77      0.75     28267
     positif       0.78      0.74      0.76     31613

   micro avg       0.76      0.76      0.76     59880
   macro avg       0.76      0.76      0.76     59880
weighted avg       0.76      0.76      0.76     59880

0.755377421509686
Time Logistic regression model: 0:00:23.267863


## Naive Bayes model

In [24]:
train2 = train.sample(100000)
start_time3 = datetime.now()
all_words = []
NUM_FEATURES = 5000
for index, value in train2.tweet.iteritems():
    if value not in all_words:
        all_words += value
top_features = [x[0] for x in nltk.FreqDist(all_words).most_common(NUM_FEATURES)]
df=pd.DataFrame.from_dict(nltk.FreqDist(all_words),orient='index')
df.sort_values(by=0, ascending=False).reset_index().head(10)
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (w in words)
    return features
featuresets = []
for index, row in train.iterrows():
    featuresets.append((build_features(row['tweet']), row['polarity']))
print(featuresets[:2])

KeyboardInterrupt: 

In [ ]:
train_set, test = train_test_split(featuresets, test_size=0.2)
classifier = NaiveBayesClassifier.train(train_set)
print(classifier.show_most_informative_features(n=10))
print(nltk.classify.accuracy(classifier, test))
# score 73,56% pour sample 100.000 avec stem nltk
# score 73,88% avec sample 100.000 avec stem spacy 

In [ ]:
print(classification_report(classifier, test))
end_time3 = datetime.now()
print('Time Naive Bayes model: {}'.format(end_time3 - start_time3))

# Step 2 : applying final model on API twitter extraction 

In [ ]:
greve_twitter=pd.read_csv('greve_twitter_final.csv',index_col=[0])
greve_twitter.head()

In [ ]:
X_cv2 = cv1_new.transform(greve_twitter['tweet_processed2'])
greve_twitter['polarity']=logreg.predict(X_cv2)
greve_twitter.head()

In [ ]:
greve_twitter.to_csv('greve_twitter_class.csv')